## EDA for Hours Spent on Labs Survey

In [44]:
suppressPackageStartupMessages(library(tidyverse))

This analysis will investigate the following question:

*How does the number of times a Master of Data Science (MDS) student goes to office hours affect the average number of hours they spend working on labs per week?*

based on the results collected from this [survey](https://ubc-mds.slack.com/archives/C24HU8X0W/p1554330074049900). We stored our raw data in this [repo]().

In [39]:
# Load survey data
rawdata = read.csv("Hours_Spent_On_Labs_Survey_Final.csv") 

In [45]:
# Data cleaning
raw_df = data.frame(rawdata)
row_to_remove = c(1:2)
col_to_remove = c(1:18, 20, 24, 29)
df = raw_df[-row_to_remove, -col_to_remove]
names(df) <- c("attend_OH", "lab_hours", "group", "academic", "yrs_out_school", "program", "stat", "optional")
df <- df %>% mutate(group = as.factor(group), academic = as.factor(academic)) %>% select (lab_hours, everything())

In [46]:
# write.csv(df, file = "clean_data.csv")